<a href="https://colab.research.google.com/github/Guidin007/Project_data/blob/main/car_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
asinow_car_price_dataset_path = kagglehub.dataset_download('asinow/car-price-dataset')

print('Data source import complete.')


# Car Price Data Analysis

In [ ]:
import pandas as pd
import zipfile as zip
import subprocess
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import numpy as np
import matplotlib.pyplot as plt

kaggle = True

In [ ]:

def get_kaggle_dataset(dataset):
    """
    Downloads a dataset from Kaggle using the Kaggle API.

    This function executes the `kaggle datasets download` command to fetch
    the specified dataset from Kaggle. It handles errors gracefully and
    prints relevant messages based on the download status.

    unzio the compress file into data folder in the project folder

    Args:
        dataset (str): The Kaggle dataset identifier in the format
                      "owner/dataset-name" (e.g., "asinow/car-price-dataset").

    Raises:
        subprocess.CalledProcessError: If the command execution fails,
                                       an error message is displayed.

    Example:
        get_kaggle_dataset("asinow/car-price-dataset")
    """
    try:
        # execute the kaggle commando
        result = subprocess.run(
            ["kaggle", "datasets", "download", "-d", dataset],
            check=True,
            capture_output=True,
            text=True
        )
        print("✅ Dataset donwloaded with succesfully!")
        print(result.stdout)
        print("Uncompress data file in data folder")

        #unzio file
        with zip.ZipFile(dataset.split('/')[-1]+".zip", 'r') as zip_ref:
            zip_ref.extractall("data")

    except subprocess.CalledProcessError as e:
        print("❌ Error descargando dataset ->", e.stderr)


# Download and read kaggle dataset

In [ ]:
# Llamada a la función con el dataset de ejemplo
if kaggle:
    path = '/kaggle/input/car-price-dataset/car_price_dataset.csv'
else:
    import kaggle as kg
    path = './data/car_price_dataset.csv'
    get_kaggle_dataset("asinow/car-price-dataset")

df_data = pd.read_csv(path, sep=',')

In [ ]:
df_data

In [ ]:
df_data.info()

In [ ]:
df_data.describe()

In [ ]:
df_data.isna().count()

In [ ]:
#It’s important to know the brand, model, mileage, and whether the price is high, similar to the top models of that brand.
brand_counts = df_data[['Brand', 'Model', 'Mileage', 'Price']].value_counts()
brand_counts = brand_counts.sort_values(ascending=False)
print(brand_counts)


In [ ]:
# Group by brand and calculate the average price
top_brands = df_data.groupby('Brand') ['Price'].mean().sort_values(ascending=False)

# Seleccionar el top 10
top_10_brands = top_brands.head(10)

# Mostrar resultados
print(top_10_brands)

In [ ]:
# Group by Brand and Model calculate the max price
top_brands = df_data.groupby(['Brand' ,'Model', 'Transmission']) ['Price'].max().sort_values(ascending=False)

# Seleccionar el top 10
top_10_brands = top_brands.head(10)

# Mostrar resultados
print(top_10_brands)

In [ ]:
# Create the pie chart
plt.figure(figsize=(8, 8))  # Adjust the size of the chart
top_10_brands.plot.pie(autopct='%1.1f%%', startangle=90, cmap='tab10')

# Set title and remove the y-axis label
plt.title("Top 10 Brands and Models with Higher Price")
plt.ylabel('')  # Hides the Y-axis label

# Show grafic pie chart
plt.show()

# Analysis of pie chart graphs
The dataset you provided shows the maximum price of certain car models from different brands with automatic transmission. Here are some key insights:

## 1️⃣ Price Analysis
Toyota Corolla has the highest price (18,301) among the listed models.
Audi A3 and Ford Explorer have very similar prices, which suggests that both luxury sedans (Audi) and SUVs (Ford) can have comparable values.
BMW 5 Series (17,386) and Mercedes C-Class (17,614) are among the lowest in this list, which is interesting because they are premium brands. This could indicate that the listed versions have less equipment or are older models.

## 2️⃣ Comparison Between Car Segments
Luxury vs. Mainstream Brands:

Audi, Mercedes, and BMW are included, but they don’t dominate the highest prices.
Mainstream brands like Toyota, Honda, and Ford also have high-priced models.
Sedans vs. SUVs:

Sedans: Corolla, A3, Accord, Elantra, Malibu, C-Class, 5 Series.
SUVs: CR-V, Explorer.
INTERESTING FACT: Sedans dominate this list, suggesting that SUVs are not among the most expensive in this case.

## 3️⃣ Differences Between Brands
Toyota has the most expensive model on the list (Corolla), which might be due to a hybrid or a highly equipped version.
Honda has two models in the ranking (Accord and CR-V), indicating that it competes in multiple segments with relatively high prices.
Audi and Mercedes are present, showing that premium cars appear on the list but don’t necessarily dominate the top spots.

## 4️⃣ Questions for Deeper Analysis
🔹 What model years are these cars? Prices can vary significantly based on the year and version.
🔹 Are these market prices for new or used cars? If they are used cars, we could analyze depreciation.
🔹 How do these prices compare to manual or hybrid versions?




# Reponse the last quiestion point 4

In [ ]:
 # What model years are these cars?
# Group by Brand and Model calculate the max price
top_brands = df_data.groupby(['Brand' ,'Model', 'Year']) ['Price'].max().sort_values(ascending=False)

# Select the top 10
Year_brands = top_brands.head(10)

# Show result
print(Year_brands)

##Are these market prices for new or used cars?
###Response: The last result shows that the cars are from different years. My conclusion is that they are secondhand or not new, If they are used cars, we could analyze depreciation.

In [ ]:
# How do these prices compare to manual or hybrid versions?
# Group by Brand, Model, Transmission calculate the max price
top_brands = df_data.groupby(['Brand' ,'Model', 'Transmission']) ['Price'].max().sort_values(ascending=False)

# Select the top 10
Transmission_brands = top_brands.head(20)

# Show result
print(Transmission_brands)

# EDA Cars

In [ ]:
df_corr = df_data[['Year',
                   'Engine_Size',
                   'Mileage',
                   'Doors',
                   'Owner_Count',
                   'Price']].corr(method='pearson')

fig = go.Figure(go.Heatmap(x = df_corr.columns,
                           y = df_corr.columns,
                           z = df_corr.values.tolist(),
                           colorscale = 'rdbu',
                           zmin = -1,
                           zmax = 1))

fig.update_layout(width = 800,
                  height = 700)

if kaggle:
    fig.show(renderer='iframe_connected')
else:
    fig.show()

In [ ]:
# Knowing what is the top Brand Grafic

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Seleccionar características (X) y variable objetivo (y)
X = df_data.drop(columns=["Price"])  # Variables predictoras
y = df_data["Price"]  # Variable objetivo

# Identificar columnas categóricas
categorical_cols = X.select_dtypes(include=["object"]).columns

# Convertir variables categóricas a numéricas usando Label Encoding
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le  # Guardamos el encoder por si lo necesitamos después

# Dividir en conjunto de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Mostrar tamaño de los conjuntos
X_train.shape, X_test.shape


In [ ]:
X

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Inicializar el modelo de regresión lineal
model = LinearRegression()

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el modelo
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Mostrar resultados
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"R²: {r2:.4f}")


In [ ]:
# Calcular errores (residuos)
errors = y_test - y_pred

# Crear histograma interactivo de los errores
fig = px.histogram(errors, nbins=50, title="Distribución de Errores (Residuos)")
if kaggle:
    fig.show(renderer='iframe_connected')
else:
    fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=y_test, y=y_pred, mode="markers", name="Predicción vs Real"))
fig.add_trace(go.Scatter(x=y_test, y=y_test, mode="lines", name="Línea Ideal", line=dict(color="red")))

fig.update_layout(title="Comparación: Precio Real vs Predicho",
                  xaxis_title="Precio Real",
                  yaxis_title="Precio Predicho")

if kaggle:
    fig.show(renderer='iframe_connected')
else:
    fig.show()